In [60]:
import pandas as pd
import numpy as np
import json
import pickle

from tqdm import tqdm_notebook

from datetime import datetime

In [61]:
with open('../data/user_features.json', 'r') as f:
    user_features = json.loads(f.read())

In [138]:
with open('../models/big_model_scoring', 'rb') as f:
    big_model_scoring = pickle.loads(f.read())
    
with open('../models/small_model_scoring', 'rb') as f:
    small_model_scoring = pickle.loads(f.read())

In [63]:
skl_dol = pd.read_excel('../data/Реестр СКЛ_ДОЛ.xlsx')

In [64]:
skl_dol.head()

,№ п/п,Вид путевки,Предприятие,Таб. номер (заявителя),ФИО сотрудника (заявителя),Должность,Подразделение,Город вылета/возвращения,Дисциплинарные взыскания,ФИО гостя,"Пол (м, ж)",Дата рождения,Дата заезда,Дата выезда,№ заявки,Дата подачи
0,1,ДОЛ,Синие горы,3.335899e+08,Сюсин Макар Сократович,Ведущий инженер,Участок по эксплуатации водоподготовительной у...,Тобольск,1,NaN,NaN,NaN,24.06.2019,14.07.2019,174932.0,2018-12-10
1,2,ДОЛ,Гундабад,1.050101e+09,Задкова Яна Данииловна,Ведущий инженер-проектировщик,Отдел строительного проектирования,Тюмень,0,Задкова Никита Эдуардович,М,2006-12-14 00:00:00,01.06.2019,21.06.2019,175272.0,2018-12-10
2,3,ДОЛ,Лесные эдьфы,4.000460e+08,Максимушкина Аза Филипповна,Начальник производства,Руководители,Красноярск,0,Максимушкина Владимир Денисович,М,2006-04-05 00:00:00,17.07.2019,06.08.2019,175352.0,2018-12-10
3,4,ДОЛ,Лесные эдьфы,4.000460e+08,Максимушкина Аза Филипповна,NaN,NaN,NaN,0,Максимушкина Дмитрий Денисович,М,2007-12-08 00:00:00,17.07.2019,06.08.2019,175352.0,2018-12-10
4,5,ДОЛ,Одинокая гора,3.002830e+08,Погодин Парфен Проклович,Ведущий инженер,Отдел капитального строительства и ремонтов зд...,Нижний Новгород,0,Погодин Юлия Александровна,Ж,2004-11-17 00:00:00,17.07.2019,06.08.2019,175647.0,2018-12-11


In [65]:
skl_dol.drop(index=skl_dol[np.isnan(skl_dol['Таб. номер (заявителя)'])].index, inplace=True)

In [66]:
skl_dol.sort_values('Дата подачи', inplace=True)

In [67]:
skl_dol = skl_dol[skl_dol['Дисциплинарные взыскания'] == 0]

In [68]:
skl_dol.drop(index=skl_dol[skl_dol['Дата заезда'] == '24.06.2019'].index, inplace=True)

In [69]:
skl_dol[skl_dol['Дата заезда'] == '17.07.2019'] = datetime(2019, 7, 17)

In [70]:
skl_dol[skl_dol['Дата заезда'] == '01.06.2019'] = datetime(2019, 6, 1)
skl_dol[skl_dol['Дата заезда'] == '09.08.2019'] = datetime(2019, 8, 9)

In [71]:
skl_dol['Дата заезда'].value_counts()

2019-09-07    2751
2019-05-05    1691
2019-07-17    1645
2019-08-01    1430
2019-06-01    1228
2019-08-16     920
2019-07-02     901
2019-06-16     785
2019-10-07     770
2019-09-22     766
2019-04-20     714
2019-03-21     550
2019-04-05     482
2019-10-22     387
2019-08-09     365
2019-11-06     188
2018-11-07       5
Name: Дата заезда, dtype: int64

In [72]:
date_mapper = {
    datetime(2019, 9, 7): 11,
    datetime(2019, 5, 5): 4,
    datetime(2019, 7, 17): 8,
    datetime(2019, 8, 1): 9,
    datetime(2019, 6, 1): 5,
    datetime(2019, 8, 16): 10,
    datetime(2019, 7, 2): 7,
    datetime(2019, 6, 16): 6,
    datetime(2019, 10, 7): 13,
    datetime(2019, 9, 22): 12,
    datetime(2019, 4, 20): 3,
    datetime(2019, 3, 21): 1,
    datetime(2019, 4, 5): 5,
    datetime(2019, 10, 22): 14,
    datetime(2019, 8, 9): 9,
    datetime(2019, 11, 6): 15,
    datetime(2019, 11, 7): 15,
}

In [88]:
good_user_ids = list(filter(lambda x: type(x) == np.float, skl_dol['Таб. номер (заявителя)'].unique().tolist()))

In [89]:
inter = np.intersect1d(list(user_features.keys()), good_user_ids)

In [94]:
inter = inter.astype(np.float)

In [96]:
user_change = {
    'user_id': [],
    'change_id': []
}

for user, group in tqdm_notebook(skl_dol[skl_dol['Таб. номер (заявителя)'].isin(inter)].groupby('Таб. номер (заявителя)')):
    change_id = None
    for d in group['Дата заезда'].values:
        try:
            change_id = date_mapper[d]
        except:
            continue
            
    if change_id != None:
        user_change['user_id'].append(user)
        user_change['change_id'].append(change_id)

HBox(children=(IntProgress(value=0, max=4186), HTML(value='')))

In [98]:
user_change = pd.DataFrame(user_change)

In [102]:
user_change.change_id.value_counts()

11    848
4     511
9     389
5     369
10    252
7     247
13    241
8     238
12    237
3     224
6     224
1     185
14    113
15     60
Name: change_id, dtype: int64

In [140]:
user_change.to_csv('../data/user_change.csv', index=False)

In [109]:
user_features['433602817.0']

{'age': 48.0,
 'divorced': 0.0,
 'exp_years': 18.493378995433787,
 'good_job': 0.0,
 'holiday': 0.0,
 'lone_mother': 0.0,
 'max_dep_age': 73.0,
 'max_dep_exp': 24.42214611872146,
 'mean_dep_age': 43.93860147811257,
 'mean_dep_exp': 11.181239340534397,
 'mean_divorced': 0.11881750994883457,
 'mean_good_job': 0.017055144968732235,
 'mean_holiday': 0.06083001705514497,
 'mean_lone_mother': 0.009664582148948267,
 'mean_motivation': 0.0,
 'mean_no_mirrage': 0.18078453666856167,
 'mean_of_mirrage': 0.6679931779420125,
 'mean_prize_1': 0.0,
 'mean_prize_2': 0.0,
 'mean_prize_3': 0.0,
 'mean_prize_4': 0.0,
 'mean_prize_5': 0.0,
 'mean_prize_6': 0.0,
 'mean_prize_7': 0.0,
 'mean_unof_mirrage': 0.010801591813530414,
 'mean_widow': 0.01137009664582149,
 'min_dep_age': 20.0,
 'min_dep_exp': 0.03287671232876712,
 'motivation': 0.0,
 'no_mirrage': 0.0,
 'of_mirrage': 1.0,
 'opek_child_mean_boy_age': 0.0,
 'opek_child_mean_girl_age': 0.0,
 'opek_child_num_boy': 0.0,
 'opek_child_num_girl': 0.0,
 'own

In [111]:
input_df = {}

for key in user_features['433602817.0'].keys():
    input_df[key] = []
    
for u in user_change.user_id.values:
    for key in input_df.keys():
        input_df[key].append(user_features[str(float(u))][key])

In [112]:
input_df = pd.DataFrame(input_df)

In [135]:
scores_1 = [p[1] for p in big_model_scoring.predict_proba(input_df)]

In [129]:
normal_columns = [
    'exp_years',
    'good_job',
    'holiday',
    'motivation',
    'prize_1',
    'prize_2',
    'prize_3',
    'prize_4',
    'prize_5',
    'prize_6',
    'prize_7'
]

new_normal_data = {}

for c in normal_columns:
    new_normal_data[c] = input_df[c].values
    
new_normal_data = pd.DataFrame(new_normal_data)

In [131]:
scores_2 = [p[1] for p in small_model_scoring.predict_proba(new_normal_data)]